In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import rbf_kernel
from numpy.linalg import eig, inv
from sklearn.svm import SVC

In [2]:
# === CONFIG ===
DATASET_PATH = r"E://Projects//paddydisease"
IMG_SIZE = 64       # Smaller to reduce memory, you can set 128 if GPU allows
MAX_PER_CLASS = 450
SEED = 42
N_COMPONENTS = 12    # KLDA projection size
GAMMA = 1e-5         # RBF kernel parameter

In [3]:
# === Load Images and Labels ===
def load_data(path, img_size):
    X, y = [], []
    for label in sorted(os.listdir(path)):
        class_path = os.path.join(path, label)
        if not os.path.isdir(class_path):
            continue
        images = os.listdir(class_path)[:MAX_PER_CLASS]
        for img_name in tqdm(images, desc=f"Loading {label}"):
            img_path = os.path.join(class_path, img_name)
            try:
                img = cv2.imread(img_path, cv2.IMREAD_COLOR)
                img = cv2.resize(img, (img_size, img_size))
                X.append(img.flatten())
                y.append(label)
            except:
                continue
    return np.array(X), np.array(y)

In [4]:

# === KLDA Computation ===
def compute_klda(X, y, gamma=1e-5, n_components=12):
    print("📌 Computing centered RBF kernel...")
    K = rbf_kernel(X, X, gamma=gamma).astype(np.float32)
    N = K.shape[0]
    classes = np.unique(y)

    one_N = np.ones((N, N), dtype=np.float32) / N
    Kc = K - one_N @ K - K @ one_N + one_N @ K @ one_N

    Sb = np.zeros((N, N), dtype=np.float32)
    Sw = np.zeros((N, N), dtype=np.float32)
    mean_total = np.mean(Kc, axis=0)

    for c in classes:
        idx = np.where(y == c)[0]
        Kc_class = Kc[idx]
        mean_class = np.mean(Kc_class, axis=0)
        n_c = len(idx)
        mean_diff = (mean_class - mean_total).reshape(-1, 1)
        Sb += n_c * (mean_diff @ mean_diff.T)
        for i in idx:
            diff = (Kc[i] - mean_class).reshape(-1, 1)
            Sw += diff @ diff.T

    eigvals, eigvecs = eig(inv(Sw + 1e-5 * np.eye(N)) @ Sb)
    top_indices = np.argsort(eigvals.real)[::-1][:n_components]
    eigvecs_top = eigvecs[:, top_indices].real
    X_klda = Kc @ eigvecs_top
    return X_klda


In [5]:
# === Main Execution ===
print("\n📦 Loading raw pixel data...")
X, y = load_data(DATASET_PATH, IMG_SIZE)
print(f"✅ Loaded {len(X)} samples with shape {X.shape[1]} features")


📦 Loading raw pixel data...


Loading normal: 100%|██████████| 450/450 [00:00<00:00, 727.51it/s]
Loading output_cascade_rcnn: 0it [00:00, ?it/s]
Loading yellow_stem_borer: 100%|██████████| 450/450 [00:00<00:00, 786.33it/s]

✅ Loaded 5850 samples with shape 12288 features


In [6]:
# === Encode Labels ===
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [7]:
# === Normalize Features ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
# === KLDA Transform ===
print("\n📈 Applying KLDA...")
X_klda = compute_klda(X_scaled, y_encoded, gamma=GAMMA, n_components=N_COMPONENTS)
print("✅ KLDA projection shape:", X_klda.shape)


📈 Applying KLDA...
📌 Computing centered RBF kernel...
✅ KLDA projection shape: (5850, 12)


In [9]:
# === Train/Test Split ===
X_train, X_test, y_train, y_test = train_test_split(X_klda, y_encoded, test_size=0.2,
                                                    stratify=y_encoded, random_state=SEED)

In [10]:
# === Classifier ===
print("\n🚀 Training classifier (SVM)...")
clf = SVC(kernel='rbf', C=10)
clf.fit(X_train, y_train)


🚀 Training classifier (SVM)...


,C,10
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [11]:
# === Evaluation ===
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\n✅ Final KLDA Accuracy (No Deep Features): {round(acc * 100, 2)}%")
print(classification_report(y_test, y_pred, target_names=le.classes_))


✅ Final KLDA Accuracy (No Deep Features): 98.12%
                          precision    recall  f1-score   support

   bacterial_leaf_blight       1.00      0.99      0.99        90
   bacterial_leaf_streak       0.90      0.96      0.92        90
bacterial_panicle_blight       0.99      0.98      0.98        90
        black_stem_borer       1.00      1.00      1.00        90
                   blast       0.92      0.96      0.94        90
              brown_spot       0.99      0.98      0.98        90
            downy_mildew       0.99      1.00      0.99        90
                   hispa       0.98      0.96      0.97        90
             leaf_roller       1.00      0.97      0.98        90
                  normal       1.00      1.00      1.00        90
                  tungro       1.00      1.00      1.00        90
        white_stem_borer       1.00      0.98      0.99        90
       yellow_stem_borer       1.00      1.00      1.00        90

                accuracy